# Fine tunning for flower dataset

In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch import optim
from PIL import Image
from collections import OrderedDict

## Download data

In [ ]:
!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz

In [ ]:
import tarfile
def members(tf, folder):
  paths = [folder + '/' + str(i) + "/" for i in range(1, 6)]
  for member in tf.getmembers():
    for p in paths:
      if member.path.startswith(p):
        yield member

with tarfile.open("flower_data.tar.gz") as tar:
    tar.extractall(members=members(tar, "train"))

with tarfile.open("flower_data.tar.gz") as tar:
    tar.extractall(members=members(tar, "valid"))

with tarfile.open("flower_data.tar.gz") as tar:
    tar.extractall(members=members(tar, "test"))


In [ ]:
# setup directory
train_dir = 'train'
valid_dir = 'valid'
test_dir = 'test'

In [ ]:
image_path = train_dir + '/1/image_06734.jpg'
img = Image.open(image_path)
img

## Load the data

### Transform data

The dataset is split into three parts, training, validation, and testing. For the training, you’ll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You’ll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model’s performance on data it hasn’t seen yet. For this you don’t want any scaling or rotation transformations, but you’ll need to resize then crop the images to the appropriate size.

The pre-trained networks you’ll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you’ll need to normalize the means and standard deviations of the images to what the network expects. For the means, it’s [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1.

Below, we apply the following transformations:

*   ToTensor: Converts the image to a pytorch tensor
*   RandomHorizontalFlip: Flips the image to give more variety to the network
*   RandomReziedCrop: Crops the image to size 224 x 224 pixels
*   Normalize: Sets the color channel for each pixel to somewhere between -1 and 1.





In [ ]:
###Transform data
data_transforms = {
    'training' : transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),transforms.RandomRotation(30),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], 
                                                         [0.229, 0.224, 0.225])]),
                                                            
    'validation' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])]),

    'testing' : transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])
}


# TODO: Load the datasets with ImageFolder
image_datasets = {
    'training' : datasets.ImageFolder(train_dir, transform=data_transforms['training']),
    'testing' : datasets.ImageFolder(test_dir, transform=data_transforms['testing']),
    'validation' : datasets.ImageFolder(valid_dir, transform=data_transforms['validation'])
}

### Load dataset

In [ ]:
# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'training' : torch.utils.data.DataLoader(image_datasets['training'], batch_size=64, shuffle=True),
    'testing' : torch.utils.data.DataLoader(image_datasets['testing'], batch_size=64, shuffle=False),
    'validation' : torch.utils.data.DataLoader(image_datasets['validation'], batch_size=64, shuffle=True)
}


## Training

In [ ]:
def train(model, epochs, learning_rate, criterion, optimizer, training_loader, validation_loader):
    
    model.train() # Puts model into training mode
    print_every = 40
    steps = 0
    use_gpu = False
    
    # Check to see whether GPU is available
    if torch.cuda.is_available():
        use_gpu = True
        model.cuda()
    else:
        model.cpu()
    
    # Iterates through each training pass based on #epochs & GPU/CPU
    for epoch in range(epochs):
        running_loss = 0
        for inputs, labels in iter(training_loader):
            steps += 1

            if use_gpu:
                inputs = Variable(inputs.float().cuda())
                labels = Variable(labels.long().cuda()) 
            else:
                inputs = Variable(inputs)
                labels = Variable(labels) 

            # Forward and backward passes
            optimizer.zero_grad() # zero's out the gradient, otherwise will keep adding
            output = model.forward(inputs) # Forward propogation
            loss = criterion(output, labels) # Calculates loss
            loss.backward() # Calculates gradient
            optimizer.step() # Updates weights based on gradient & learning rate
            running_loss += loss.item()

            if steps % print_every == 0:
                validation_loss, accuracy = validate(model, criterion, validation_loader)

                print("Epoch: {}/{} ".format(epoch+1, epochs),
                        "Training Loss: {:.3f} ".format(running_loss/print_every),
                        "Validation Loss: {:.3f} ".format(validation_loss),
                        "Validation Accuracy: {:.3f}".format(accuracy))

## Validate

In [ ]:
def validate(model, criterion, data_loader):
    model.eval() # Puts model into validation mode
    accuracy = 0
    test_loss = 0
    
    for inputs, labels in iter(data_loader):
        if torch.cuda.is_available():
            inputs = Variable(inputs.float().cuda(), volatile=True)
            labels = Variable(labels.long().cuda(), volatile=True) 
        else:
            inputs = Variable(inputs, volatile=True)
            labels = Variable(labels, volatile=True)

        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        ps = torch.exp(output).data 
        equality = (labels.data == ps.max(1)[1])
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss/len(data_loader), accuracy/len(data_loader)

# Activity 1


Create your own model and evaluate its performance

(Look at activity from CNN class)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(256*28*28, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 5))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = CNN()
epochs = 9
learning_rate = 0.001
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train(model, epochs, learning_rate, criterion, optimizer, dataloaders['training'], dataloaders['validation'])

# Transfer learning

In [ ]:
model = models.vgg16(pretrained=True)
model

In the classifier, the first line (0): Linear(in_features=25088) indicates that it’s expecting 25088 inputs into the first layer. When we define our own classifier below, we will keep this input size. However, we will need to adjust the last output to match the number of categories we have. In this case 5 flower species. We use ReLU activation functions at each hidden layer, and then apply a Softmax loss function to calculate error.

In [ ]:
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)), # First layer
                          ('relu', nn.ReLU()), # Apply activation function
                          ('fc2', nn.Linear(4096, 5)), # Output layer
                          ('output', nn.LogSoftmax(dim=1)) # Apply loss function
                          ]))

We don’t want to udpate the weights of our pretrained model, just the classifier. So we can use the following code:

In [ ]:
for param in model.parameters():
    param.requires_grad = False

Next, we replace the classifier in the model with the one we just built.

In [ ]:
model.classifier = classifier

In [ ]:
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
train(model, epochs, learning_rate, criterion, optimizer, dataloaders['training'], dataloaders['validation'])

# Activites 2.


**Try doing fine-tunning instead of freezing. Does it increase performance?** You need to use a much smaller learning rate for Adam

In [ ]:
# Your code below

#Activity 3

**Try with Inception3 instead of VGG16. Do you see differences in performance?**

In [ ]:
# Your code below